In [2]:
#Importing Modules
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import statsmodels.api as sm

In [3]:
#Loading the dataset
df = pd.read_excel('Online Retail.xlsx')

#Displaying the first few rows of the dataframe
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


#### Data Preprocessing

In [4]:
#Checking for missing values
print(df.isnull().sum())

#Dropping rows with missing CustomerID as they are essential for our analysis
df = df.dropna(subset=['CustomerID'])

#Dropping duplicates
df = df.drop_duplicates()

#Displaying summary statistics to understand the data better
df.describe()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


,Quantity,UnitPrice,CustomerID
count,401604.000000,401604.000000,401604.000000
mean,12.183273,3.474064,15281.160818
std,250.283037,69.764035,1714.006089
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13939.000000
50%,5.000000,1.950000,15145.000000
75%,12.000000,3.750000,16784.000000
max,80995.000000,38970.000000,18287.000000


In [5]:
#Converting InvoiceDate to datetime for easy data manipulation
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

#Checking data types
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [6]:
#Calculating total revenue for each transaction
df['TotalRevenue'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalRevenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [7]:
#Aggregating data by CustomerID to get total revenue and number of transactions per customer
customer_data = df.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',  # Number of unique transactions
    'TotalRevenue': 'sum'    # Total revenue
}).reset_index()

customer_data.rename(columns={'InvoiceNo': 'NumTransactions'}, inplace=True)
customer_data.head()


,CustomerID,NumTransactions,TotalRevenue
0,12346.0,2,0.00
1,12347.0,7,4310.00
2,12348.0,4,1797.24
3,12349.0,1,1757.55
4,12350.0,1,334.40


#### Simulating Campaigns for Analysis

In [8]:
# Defining campaign period
campaign_start_date = '2011-11-01'
campaign_end_date = '2011-11-30'

#Creating a new column to indicate whether a transaction was made during the period
df['DuringCampaign'] = df['InvoiceDate'].between(campaign_start_date, campaign_end_date)

#Aggregating data by campaign period
campaign_data = df.groupby('DuringCampaign').agg({
    'InvoiceNo': 'nunique',
    'CustomerID': 'nunique',
    'TotalRevenue': 'sum'
}).reset_index()

campaign_data.rename(columns={
    'InvoiceNo': 'NumTransactions',
    'CustomerID': 'NumCustomers',
    'TotalRevenue': 'TotalRevenue'
}, inplace=True)

campaign_data.head()

,DuringCampaign,NumTransactions,NumCustomers,TotalRevenue
0,False,19238,4106,7192140.764
1,True,2952,1666,1086378.660


In [ ]:
#Saving the cleaned and prepared data to a CSV file
customer_data.to_csv('customer_data.csv', index=False)
campaign_data.to_csv('campaign_data.csv', index=False)

#### A/B Testing

For A/B testing, we need to define control and treatment groups. Let's assume the treatment group includes transactions during the campaign period and the control group includes transactions before the campaign period.

In [9]:
#Creating control and treatment groups
control_group = df[df['InvoiceDate'] < campaign_start_date]
treatment_group = df[df['InvoiceDate'].between(campaign_start_date, campaign_end_date)]

#Aggregating total revenue for each group
control_revenue = control_group['TotalRevenue'].sum()
treatment_revenue = treatment_group['TotalRevenue'].sum()

#Calculating mean revenue per transaction for each group
control_mean_revenue = control_group['TotalRevenue'].mean()
treatment_mean_revenue = treatment_group['TotalRevenue'].mean()

#Performing t-test to compare the means
t_stat, p_value = stats.ttest_ind(control_group['TotalRevenue'], treatment_group['TotalRevenue'])

print(f"Control Group Mean Revenue: {control_mean_revenue}")
print(f"Treatment Group Mean Revenue: {treatment_mean_revenue}")
print(f"T-Statistic: {t_stat}")
print(f"P-Value: {p_value}")

Control Group Mean Revenue: 21.282430463458176
Treatment Group Mean Revenue: 17.507270559037778
T-Statistic: 4.020812376085308
P-Value: 5.800890810424956e-05


***Statistical Significance:***
The p-value is much less than the commonly used significance level of 0.05. This indicates that the difference in mean revenue between the control and treatment groups is statistically significant and not likely due to random variation.

***Impact on Revenue:***
The control group had a higher mean revenue per transaction (21.28) than the treatment group (17.51).
Despite the campaign efforts, revenue per transaction was lower during the campaign period than before the campaign.

***Practical Implications***
- Campaign Analysis: The campaign had a significant impact, but the effect was negative regarding revenue per transaction in the simulation.
- Actionable Insights: This insight suggests that the simulated campaign may not have been effective in generating higher revenue and might need to be reevaluated. In a real-case scenario, the campaign strategy, targeting, timing, and execution should be examined to understand why the campaign did not perform as expected.


#### Difference-in-Differences

For difference-in-differences (DiD), we'll compare pre- and post-campaign performance for both groups.

In [10]:
#Defining pre-campaign period
pre_campaign_end_date = '2011-10-31'

#Creating pre and post campaign groups
pre_campaign = df[df['InvoiceDate'] <= pre_campaign_end_date]
post_campaign = df[df['InvoiceDate'] > campaign_end_date]

#Aggregating data for DiD analysis
pre_campaign_grouped = pre_campaign.groupby(['CustomerID']).agg({'TotalRevenue': 'sum'}).reset_index()
post_campaign_grouped = post_campaign.groupby(['CustomerID']).agg({'TotalRevenue': 'sum'}).reset_index()

#Merging pre and post campaign data
did_df = pre_campaign_grouped.merge(post_campaign_grouped, on='CustomerID', suffixes=('_pre', '_post'))

#Calculating difference
did_df['Revenue_Diff'] = did_df['TotalRevenue_post'] - did_df['TotalRevenue_pre']

#Mean difference
mean_diff_pre = pre_campaign_grouped['TotalRevenue'].mean()
mean_diff_post = post_campaign_grouped['TotalRevenue'].mean()

#Difference-in-differences
did = mean_diff_post - mean_diff_pre

print(f"Mean Difference Pre-Campaign: {mean_diff_pre}")
print(f"Mean Difference Post-Campaign: {mean_diff_post}")
print(f"Difference-in-Differences: {did}")

Mean Difference Pre-Campaign: 1694.0575216195978
Mean Difference Post-Campaign: 497.36437499999903
Difference-in-Differences: -1196.6931466195988


***Revenue Decrease:*** The negative DiD value suggests that there was a substantial decrease in average revenue per customer during the campaign period compared to the pre-campaign period, similar to the A/B result 

***Campaign Effectiveness:*** The campaign appears to have had a negative impact on revenue, leading to a significant drop in customer spending.

#### Regression Analysis

In [11]:
# Create a binary variable for the campaign period
df['Campaign'] = np.where(df['InvoiceDate'].between(campaign_start_date, campaign_end_date), 1, 0)

#Initialising a regression model
X = df[['Campaign', 'Quantity', 'UnitPrice']]  # Add other control variables as needed
y = df['TotalRevenue']

#Adding a constant to the model
X = sm.add_constant(X)

#Fitting the model
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           TotalRevenue   R-squared:                       0.856
Model:                            OLS   Adj. R-squared:                  0.856
Method:                 Least Squares   F-statistic:                 7.940e+05
Date:                Wed, 29 May 2024   Prob (F-statistic):               0.00
Time:                        19:49:35   Log-Likelihood:            -2.6167e+06
No. Observations:              401604   AIC:                         5.233e+06
Df Residuals:                  401600   BIC:                         5.233e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.2830      0.281     15.235      0.0